## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
#from keras.models import Sequential
#from keras.layers import Convolution2D
#from keras.layers import MaxPooling2D
#from keras.layers import Flatten
#from keras.layers import Dense
#from keras.layers import Dropout
#from keras.layers import BatchNormalization
#from keras.datasets import cifar10
#import numpy as np
#import tensorflow as tf
#from sklearn.preprocessing import OneHotEncoder


In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis = (0,1,2,3))
        std = np.std(X_train, axis = (0, 1, 2, 3))
        X_train = (X_train-mean) / (std+1e-7)
        X_test = (X_test-mean) / (std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [4]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot = OneHotEncoder()
y_train = one_hot.fit_transform(y_train).toarray()
y_test = one_hot.transform(y_test).toarray()

/Users/jasonch/anaconda3/envs/tf_cpu/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [5]:
classifier = Sequential()

#卷積組合
classifier.add(Convolution2D(32, (3, 3), input_shape = (32, 32, 3), activation = 'relu'))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
#classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(64, (3, 3)))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size = (2,2)))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(units = 100, activation = 'relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(units = 10, activation = 'softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
#classifier.fit(x_train, y_train, batch_size = 100, epochs = 100)
classifier.fit(x_train, y_train, batch_size = 10, epochs = 10)

Train on 50000 samples
Epoch 1/10
50000/50000 [==============================] - 235s 5ms/sample - loss: 1.6678 - accuracy: 0.3830
Epoch 2/10
50000/50000 [==============================] - 218s 4ms/sample - loss: 1.2371 - accuracy: 0.5561
Epoch 3/10
50000/50000 [==============================] - 224s 4ms/sample - loss: 1.0676 - accuracy: 0.6206
Epoch 4/10
50000/50000 [==============================] - 224s 4ms/sample - loss: 0.9568 - accuracy: 0.6612
Epoch 5/10
50000/50000 [==============================] - 218s 4ms/sample - loss: 0.8651 - accuracy: 0.6944
Epoch 6/10
50000/50000 [==============================] - 224s 4ms/sample - loss: 0.7851 - accuracy: 0.7238
Epoch 7/10
50000/50000 [==============================] - 223s 4ms/sample - loss: 0.7016 - accuracy: 0.7539
Epoch 8/10
50000/50000 [==============================] - 230s 5ms/sample - loss: 0.6342 - accuracy: 0.7762
Epoch 9/10
50000/50000 [==============================] - 223s 4ms/sample - loss: 0.5691 - accuracy: 0.7991
Epoch

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [ ]:
input_example=(np.zeros(shape=(1,32,32,3)) - mean_train)/(std_train + 1e-7) 
classifier.predict(input_example)